In [1]:
import tensorflow as tf

# List available physical devices (GPUs)
physical_devices = tf.config.list_physical_devices('GPU')

# Check if there are GPUs available
if len(physical_devices) > 0:
    # Set memory growth for each GPU device
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        
    # Set the visible GPU device
    tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [ ]:
import os
import numpy as np
from music21 import converter, note, chord
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

midi_folder = 'classicalmusic'
notes = []
i = 0
# Step 1: Extract notes and chords from MIDI files
for file in os.listdir(midi_folder):
    i = i + 1
    if file.endswith(".mid") or file.endswith(".midi"):
        try:
            midi = converter.parse(os.path.join(midi_folder, file))
            print(f"Parsing {i}: {file}")
            elements = midi.flat.notes  # simple & safe

            for element in elements:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        except Exception as e:
            print(f"Failed to parse {file}: {e}")

print(f"Total notes/chords collected: {len(notes)}")

# Step 2: Prepare sequences for the model
sequence_length = 50
pitch_names = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(pitch_names)

print(f"Total patterns: {n_patterns}, Vocabulary size: {n_vocab}")

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)




🎵 Parsing: 001399bbeb84eb9cdc6ac012020da644.mid
🎵 Parsing: 01160c59c4f8b79bae3b621c22b23213.mid
🎵 Parsing: 01ecdcf9c772fb23eaecdcafe7de99af.mid
🎵 Parsing: 02a7c29c8890eb2baee07844f1f66b78.mid
🎵 Parsing: 02a9203941a4ae8a5f410964edfe1702.mid
🎵 Parsing: 02c1df7932ba5de91f955b21263df541.mid
🎵 Parsing: 02c9b449dd2835b1680e9d65a8f04c15.mid
🎵 Parsing: 030ed41e1b2465d18e21bc3f780c4d07.mid
🎵 Parsing: 041850225ecaf69a0a30dcd754c62b9a.mid
🎵 Parsing: 043a8a6a2c1c6080866bfe4f70fe52df.mid
🎵 Parsing: 04a95cee5af8122804032454836f1e16.mid
🎵 Parsing: 04f0231e08bcb10f59ce4e7b899d16cf.mid
🎵 Parsing: 050c8bfea6b8738a496cafcbc64d5e4e.mid
🎵 Parsing: 057db46ad66cedff6cb461e1584b150a.mid
🎵 Parsing: 05848fce858ee1a9bf459f37d5ba3e9c.mid
🎵 Parsing: 05959aaac644017660224b996281edb0.mid
🎵 Parsing: 05dc6e608726011d69a768029de6aaeb.mid
🎵 Parsing: 05fe9270f0dff6119a44d7ecc93209cf.mid
🎵 Parsing: 062b81756f4c9382783b3ad9440279a5.mid
🎵 Parsing: 062d257f0443fb76c2e130192d2b4786.mid
🎵 Parsing: 065b3f55211e23ea3b1a8072fed32

In [ ]:
network_input = network_input[:50000]
network_output = network_output[:50000]
network_input = network_input.astype('float16')
network_output = network_output.astype('float16')
# Slice first 50k samples for testing


# Step 3: Build LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
print(" Model built successfully!")




🧠 Model built successfully!


In [ ]:
# Step 4: Train model
model.fit(network_input, network_output, epochs=10, batch_size=16)

# Optional Step 5: Save model & data
model.save("music_model.h5")
np.save("network_input.npy", network_input)
np.save("pitch_names.npy", pitch_names)

print(" Training complete! You’re ready to generate music.")

Epoch 1/10
3125/3125 [==============================] - 101s 32ms/step - loss: 5.8634
Epoch 2/10
3125/3125 [==============================] - 100s 32ms/step - loss: 5.8096
Epoch 3/10
3125/3125 [==============================] - 101s 32ms/step - loss: 5.7332
Epoch 4/10
3125/3125 [==============================] - 100s 32ms/step - loss: 5.7070
Epoch 5/10
3125/3125 [==============================] - 77s 25ms/step - loss: 5.6883
Epoch 6/10
3125/3125 [==============================] - 50s 16ms/step - loss: 5.6709
Epoch 7/10
3125/3125 [==============================] - 60s 19ms/step - loss: 5.6445
Epoch 8/10
3125/3125 [==============================] - 60s 19ms/step - loss: 5.6232
Epoch 9/10
3125/3125 [==============================] - 60s 19ms/step - loss: 5.5995
Epoch 10/10
3125/3125 [==============================] - 60s 19ms/step - loss: 5.5773
🎉 Training complete! You’re ready to generate music.


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from music21 import instrument, note, chord, stream
import random
import sys # For error exit

# --- Configuration ---
MODEL_PATH = "music_model.h5"
INPUT_DATA_PATH = "network_input.npy"
PITCH_NAMES_PATH = "pitch_names.npy"
OUTPUT_MIDI_PATH = "generated_music_improved.mid"
NOTES_TO_GENERATE = 500
# !!! IMPORTANT: Set this to the sequence length your model was trained on !!!
# You might be able to infer this from network_input.shape[1] if loaded correctly
SEQUENCE_LENGTH = 50 # <--- Adjust this value!
# --- Sampling Temperature ---
# Experiment with values (e.g., 0.5, 0.75, 1.0, 1.2)
# Lower values = more predictable, closer to original; Higher values = more random
TEMPERATURE = 0.85

# --- Helper Function for Temperature Sampling ---
def sample_with_temperature(preds, temperature=1.0):
    """
    Samples an index from a probability array, controlling randomness with temperature.
    Args:
        preds (np.array): Array of probabilities from the model's output layer.
        temperature (float): Controls randomness. >0. Values <1 sharpen, >1 flatten.
    Returns:
        int: The sampled index.
    """
    if temperature <= 1e-8: # Treat 0 or near-zero temperature as argmax
        return np.argmax(preds)

    preds = np.asarray(preds).astype('float64') # Use float64 for numerical stability

    # Apply temperature
    # Adding epsilon prevents log(0)
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)

    # Re-normalize using softmax formula
    preds = exp_preds / np.sum(exp_preds)

    # Sample using multinomial (picks one index based on probabilities)
    # preds must sum to 1.0 +/- a tiny tolerance
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

    # --- Alternative using np.random.choice (sometimes simpler to read) ---
    # choices = np.arange(len(preds))
    # return np.random.choice(choices, p=preds)

# --- Load Model and Data ---
print("Loading model and data...")
try:
    model = load_model(MODEL_PATH)
    # Load the full dataset first to get a pattern
    network_input_full = np.load(INPUT_DATA_PATH, allow_pickle=True)
    pitch_names = np.load(PITCH_NAMES_PATH, allow_pickle=True)
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    print(f"Please ensure '{MODEL_PATH}', '{INPUT_DATA_PATH}', and '{PITCH_NAMES_PATH}' are in the correct directory.")
    sys.exit(1)
except Exception as e:
    print(f"An error occurred during loading: {e}")
    sys.exit(1)

# --- Prepare Generation ---
n_vocab = len(pitch_names)
int_to_note = dict((number, note_str) for number, note_str in enumerate(pitch_names))

# --- Validate Input Data Shape and Sequence Length ---
if len(network_input_full.shape) < 2:
     print(f"Error: '{INPUT_DATA_PATH}' has unexpected shape {network_input_full.shape}.")
     print("Expected at least 2 dimensions (samples, sequence_length).")
     sys.exit(1)

# Infer sequence length from loaded data
actual_sequence_length = network_input_full.shape[1]
if actual_sequence_length != SEQUENCE_LENGTH:
    print(f"Warning: Provided SEQUENCE_LENGTH ({SEQUENCE_LENGTH}) does not match data's sequence length ({actual_sequence_length}).")
    print(f"Using sequence length from data: {actual_sequence_length}")
    SEQUENCE_LENGTH = actual_sequence_length # Use the actual length from data

if len(network_input_full) == 0:
    print(f"Error: '{INPUT_DATA_PATH}' seems to be empty.")
    sys.exit(1)

# --- Select Starting Pattern ---
start_index = np.random.randint(0, len(network_input_full) - 1)
initial_pattern_raw = network_input_full[start_index]

# Ensure the pattern is a flat list of integers of the correct length
# Handle case where input might have an extra dimension (e.g., shape (100, 1))
if initial_pattern_raw.ndim > 1 and initial_pattern_raw.shape[-1] == 1:
    pattern = initial_pattern_raw.flatten().tolist()
else:
    pattern = initial_pattern_raw.tolist() # Assume it's already (seq_len,)

if len(pattern) != SEQUENCE_LENGTH:
     print(f"Error: Selected pattern length ({len(pattern)}) does not match required sequence length ({SEQUENCE_LENGTH}).")
     print(f"Check the structure of '{INPUT_DATA_PATH}' and the SEQUENCE_LENGTH setting.")
     sys.exit(1)

print(f"Using sequence length: {SEQUENCE_LENGTH}")
print(f"Using temperature: {TEMPERATURE}")
print(f"Generating {NOTES_TO_GENERATE} notes...")

generated_indices = [] # Store the generated *indices* first

# --- Generate Notes ---
for note_index in range(NOTES_TO_GENERATE):
    # Prepare input: reshape and normalize
    # Reshape pattern into the format the model expects (batch_size, sequence_length, features)
    # Assuming features=1 where the feature is the normalized index
    prediction_input = np.reshape(pattern, (1, SEQUENCE_LENGTH, 1))

    # !!! CRITICAL: Ensure this normalization matches your training !!!
    prediction_input = prediction_input / float(n_vocab)

    # Predict next note probabilities
    try:
        prediction = model.predict(prediction_input, verbose=0)[0] # Get prediction for the single sequence
    except Exception as e:
        print(f"\nError during model prediction (note index {note_index}): {e}")
        print("Check model compatibility, input shape, and normalization.")
        break # Stop generation on error

    # Sample the next note index using temperature
    index = sample_with_temperature(prediction, TEMPERATURE)
    generated_indices.append(index)

    # Update the pattern: append the new *integer* index and slide the window
    pattern.append(index)
    pattern = pattern[1:] # Keep the pattern length consistent

    # Optional: Progress indicator
    if (note_index + 1) % 50 == 0:
        print(f"  Generated {note_index + 1}/{NOTES_TO_GENERATE} notes...")

# --- Convert Indices to Music21 Objects ---
print("\nConverting generated indices to MIDI...")
output_notes = []
offset = 0.0 # Use float for offsets

# Map indices back to note/chord strings
generated_note_strings = [int_to_note.get(index, None) for index in generated_indices] # Use .get for safety

for element_string in generated_note_strings:
    if element_string is None:
        print("Warning: Encountered an invalid index during generation. Skipping.")
        continue

    # If it's a chord (e.g., '60.64.67') - check for '.' or if it's purely digits (representing MIDI num)
    # This check might need adjustment based on exactly how chords are stored in pitch_names
    if ('.' in element_string) or (element_string.isdigit() and '.' not in element_string): # Added check for single MIDI num
         notes_in_chord_str = element_string.split('.')
         notes_objs = []
         for pitch_val_str in notes_in_chord_str:
             try:
                 # Try converting to int (assuming MIDI numbers)
                 pitch_val = int(pitch_val_str)
                 new_note_obj = note.Note(pitch_val)
                 new_note_obj.storedInstrument = instrument.Piano() # Assign instrument
                 notes_objs.append(new_note_obj)
             except ValueError:
                 # If not int, maybe it's a pitch name? music21 handles this in Note()
                 try:
                    new_note_obj = note.Note(pitch_val_str) # Try creating note directly
                    new_note_obj.storedInstrument = instrument.Piano()
                    notes_objs.append(new_note_obj)
                 except Exception as e_inner:
                     print(f"Warning: Could not parse pitch '{pitch_val_str}' in chord '{element_string}'. Error: {e_inner}. Skipping pitch.")
                     continue # Skip this specific pitch if unparseable
         if notes_objs: # Only create chord if notes were successfully parsed
             new_chord = chord.Chord(notes_objs)
             new_chord.offset = offset
             output_notes.append(new_chord)
    # If it's a single note (e.g., 'C4', 'F#5')
    else:
        try:
            # music21 Note can handle pitch names or MIDI numbers directly
            new_note = note.Note(element_string)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        except Exception as e: # Catch broader music21 errors
             print(f"Warning: Could not parse note '{element_string}'. Error: {e}. Skipping note.")
             continue # Skip this note if unparseable

    # Increment offset for the next note/chord
    # You could make this more sophisticated by using note/chord durations later
    offset += 0.5 # Fixed advance, adjust rhythm as needed

# --- Create and Save MIDI File ---
if not output_notes:
    print("\nWarning: No valid notes were generated or converted. Cannot create MIDI file.")
else:
    midi_stream = stream.Stream(output_notes)
    try:
        midi_stream.write('midi', fp=OUTPUT_MIDI_PATH)
        print(f"\n Generated music saved as '{OUTPUT_MIDI_PATH}'")
    except Exception as e:
        print(f"\nError writing MIDI file: {e}")

# --- Print Sample Output ---
print(" Sample of generated note/chord strings:", generated_note_strings[:20])

print("\nGeneration complete.")

Loading model and data...
Using sequence length: 50
Using temperature: 0.85
Generating 500 notes...
  Generated 50/500 notes...
  Generated 100/500 notes...
  Generated 150/500 notes...
  Generated 200/500 notes...
  Generated 250/500 notes...
  Generated 300/500 notes...
  Generated 350/500 notes...
